# DeepAgents 演示：LangChain-ai 深度智能体
> 项目地址：https://github.com/langchain-ai/deepagents
>
> **本演示展示**：
> - ✅ DeepAgents 基础使用（规划工具、文件系统、子代理）
> - ✅ 集成 Qwen 模型（低成本本地调试）
> - ✅ Obsidian 助手（本地知识库搜索）

## ? 环境配置

**前置要求**：
1. 已安装 `langchain-anthropic`（DeepAgents 必需）
2. 已安装 `deepagents` 库（可编辑模式）
3. 配置 `.env` 文件（DASHSCOPE_API_KEY）

运行下面的单元格验证环境：

In [14]:
# 步骤 1: 配置 Python 路径并验证导入
import sys
from pathlib import Path

# 添加项目根目录到 Python 路径
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"✅ 项目根目录: {project_root}")

# 验证能否导入项目模块
try:
    from deepagents import create_deep_agent
    print("✅ deepagents 模块导入成功")
    
    from obsidian_assistant.obsidian_assistant import create_obsidian_assistant_v2
    from obsidian_assistant.token_counter import TokenCounter
    print("✅ 项目模块导入成功")
    print("\n🎉 环境配置完成，可以开始测试！")
except ImportError as e:
    print(f"❌ 模块导入失败: {e}")
    print("\n💡 请确保已安装依赖：")
    print("   pip install langchain-anthropic")
    print("   cd deepagents_official/libs/deepagents && pip install -e .")

✅ 项目根目录: /Users/yf/Documents/GitHub/deepagents
✅ deepagents 模块导入成功
✅ 项目模块导入成功

🎉 环境配置完成，可以开始测试！


---

## 🚀 示例 1：DeepAgents 基础调用

使用 Qwen 模型（通义千问）进行基础对话测试。

In [ ]:
from langchain_community.chat_models import ChatTongyi # Qwen 模型导入
from deepagents import create_deep_agent # DeepAgents 内置函数
from dotenv import load_dotenv # 加载环境变量

load_dotenv()

# 创建 DeepAgents，使用 Qwen 模型
model = ChatTongyi(model='qwen-turbo')
# model、tools、system_prompt 可根据需要调整
agent = create_deep_agent(
    model=model, 
    tools=[], 
    system_prompt='You are an assistant.')

# 测试调用
result = agent.invoke({"messages": [{"role": "user", "content": "你好，介绍一下自己吧。"}]})

# 展示结果（使用工具函数）
from utils import display_agent_execution, extract_final_answer
display_agent_execution(result)

🤖 Agent 执行流程

👤 用户:
   你好，介绍一下自己吧。

🤖 Agent (步骤 1):
   💬 回复: 你好！我是Qwen，是阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我能够回答问题、创作文字、编程、逻辑推理等多种任务。如果你有任何问题或需要帮助，随时告诉我！

✅ 执行完成


---

## 🔍 示例 2：Deepagents_Obsidian 助手（本地知识库搜索）

集成自定义工具和子代理，实现本地知识库智能问答。

In [16]:
# 加载 Obsidian 助手
from obsidian_assistant.obsidian_assistant import create_obsidian_assistant_v2
from obsidian_assistant.token_counter import TokenCounter
from utils import count_tokens_for_result

# 创建助手实例
assistant = create_obsidian_assistant_v2()
token_counter = TokenCounter(model="qwen-turbo")

print("✅ Obsidian 助手已就绪")

✅ Obsidian 助手已就绪


In [17]:
# 测试问题
question = "如何在 Obsidian 中创建双向链接？"

# 调用助手
token_counter.start_counting()
result = assistant.invoke({"messages": [("user", question)]})

# 统计 Token
record = count_tokens_for_result(question, result, token_counter)
token_counter.print_current_usage(record)

# 展示结果
from utils import display_agent_execution
display_agent_execution(result)


📊 Token 使用统计
🕐 时间: 2025-11-14 15:24:25
❓ 问题: 如何在 Obsidian 中创建双向链接？
⏱️  响应时间: 4.30 秒

💬 Token 详情:
  - 输入:      18
  - 输出:     523
  - 总计:     541

💰 预估成本: ¥0.0032 元

🤖 Agent 执行流程

👤 用户:
   如何在 Obsidian 中创建双向链接？

🤖 Agent (步骤 1):
   💬 回复: 在 Obsidian 中创建双向链接是一种非常有用的功能，它可以帮助你更好地组织和导航笔记。以下是创建双向链接的步骤：

1. **内部链接**：使用双方括号 `[[ ]]` 包裹笔记名称。例如，如果你有一个名为 "Example Note" 的笔记，你可以写成 `[[Example Note]]`。这会在当前笔记中创建一个指向 "Example Note" 的链接，并在 "Example Note" 中自动创建一个反向链接（参考：[[Linking notes and files/Internal links|内部链接]]）。

2. **外部链接**：使用 Markdown 格式 `[文本](URL)`。例如，`[Obsidian 官网](https://obsidian.md)` 会创建一个指向 Obsidian 官网的链接（参考：[[Editing and formatting/Basic formatting syntax|基础格式化语法]]）。

3. **双向链接**：当你在一篇笔记中创建一个内部链接到另一篇笔记时，Obsidian 会自动在目标笔记中创建一个反向链接。例如，如果你在 "Note A" 中创建了 `[[Note B]]`，那么在 "Note B" 中会自动生成一个指向 "Note A" 的链接。

### 参考来源
⭐⭐⭐ [[Linking notes and files/Internal links|内部链接]] - 核心参考
⭐⭐ [[Editing and formatting/Basic formatting syntax|基础格式化语法]] - 补充阅读

✅ 执行完成


---

## 📚 扩展阅读

**DeepAgents 核心特性**：
- 🧠 **规划工具**：`write_todos` 分解复杂任务
- 📁 **文件系统**：`ls`, `read_file`, `write_file`, `edit_file` 管理上下文
- 🤖 **子代理**：`task` 工具隔离上下文，深度执行子任务
- 💾 **长期记忆**：使用 LangGraph Store 跨对话持久化

**自定义配置**：
- `model`：切换不同 LLM（OpenAI、Claude、Qwen 等）
- `tools`：添加自定义工具（如搜索、API 调用）
- `system_prompt`：定制 Agent 行为和专业领域
- `subagents`：创建专用子代理（研究、编程、数据分析等）

**更多示例**：参见 `deepagents_official/examples/` 目录